Primeiro, vamos realizar a importação de todas as bibliotecas que usaremos.

In [1297]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, normalize, LabelEncoder
from sklearn.model_selection import KFold, train_test_split, RandomizedSearchCV, GridSearchCV, cross_val_predict, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, make_scorer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier, GradientBoostingClassifier, ExtraTreesClassifier, BaggingClassifier, AdaBoostClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, Lasso, RidgeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_selection import SelectFromModel

Agora, vamos importar nossos dados.

In [1298]:
df_test = pd.read_csv('../test.csv')
df = pd.read_csv('../train.csv')

In [1299]:
pd.set_option('display.max_columns', None) # Configuracao para mostrar todas as colunas

## Entendendo o significado de todas as colunas

É de extrema importância entender todas as colunas, por isso, vamos copiar e colar o significado delas:

- track_id: O ID único de cada música
    
- artists: Nome dos(as) artistas que performaram a música, separados por ';'
    
- album_name: Nome do álbum no qual aparece a música
    
- track_name: Nome da música
    
- duration_ms: A duração da música em milissegundos
    
- explicit: Boolean indicando se a música possui conteúdo explícito
    
- danceability: Descreve quanto uma música é "dançante" (0.0 = menos dançante, 1.0 = mais dançante)
    
- energy: Representa a intensidade e atividade de uma música (0.0 = baixa energia, 1.0 = alta energia)
    
- key: A tonalidade musical da faixa mapeada usando a notação padrão de Classe de Altura (12 notas musicais)
    
- loudness: Nível geral de volume da faixa em decibéis (dB)
    
- mode: Indica a modalidade (maior ou menor) da faixa
    
- speechiness: Detecta a presença de palavras faladas na faixa
    
- acousticness: Medida de confiança sobre se a faixa é acústica (0,0 = não acústica, 1,0 = altamente acústica)
    
- instrumentalness: Prediz se uma faixa contém vocais (0,0 = contém vocais, 1,0 = instrumental)
    
- liveness: Detecta a presença de uma audiência na gravação (0,0 = gravação em estúdio, 1,0 = performance ao vivo)
    
- valence: Mede a positividade musical transmitida por uma faixa (0,0 = negativa, 1,0 = positiva)
    
- tempo: Tempo estimado da faixa em batidas por minuto (BPM)
    
- time_signature: Assinatura de tempo estimada da faixa (de 3 a 7)
    
- track_genre: O gênero da música
    
- popularity_target: Boolean indicando se a música é popular ou não

Realizando a análise das colunas, fiquei em dúvida sobre a diferença entre a coluna 'tempo' e 'time_signature'. Por isso, realizei uma análise mais profunda no significado delas:

* tempo (Batidas por minuto):
   * O tempo representa a velocidade ou ritmo de uma faixa, medido em batidas por minuto (BPM).
   * O tempo médio é de cerca de 122 BPM, o que representa um ritmo moderado e enérgico.
   * O tempo mínimo é 0 BPM (o que pode indicar dados ausentes ou faixas muito lentas), e o máximo é cerca de 223 BPM (muito rápido).

* time_signature (Fórmula de compasso):
   * A fórmula de compasso representa a estrutura rítmica de uma faixa, indicando quantas batidas existem em cada compasso e qual valor de nota representa uma batida.
   * Os valores geralmente variam de 3 a 7, sendo 4 o mais comum (representando o compasso 4/4, que é padrão em muitos gêneros).
   * A partir das estatísticas, podemos ver que a mediana e os percentis 25 e 75 são 4, confirmando que o compasso 4/4 é realmente o mais comum.
   * O mínimo de 0 pode indicar dados ausentes ou fórmulas de compasso não convencionais.

   Primeiro dou uma olhada por cima dos dados apenas para entender quais colunas são númericas e categóricas. Também tenho a intenção de ver elas como dados reais e por isso chamo o comando `df.head(3)`

In [ ]:
df.head(3)

In [ ]:
df.info()

Agora partindo para uma análise mais profunda das colunas numéricas, começo com uma análise da estatística descritiva de cada coluna.

In [ ]:
df.describe()

Com o output acima, já é possível observar que valores iguais a 0 em 'tempo' e 'time_signature' provavelmente são missing values. Chegamos a essa conclusão porque músicas com 0 BPM não existem, mesmo muito lentas. E 'time_sigture' já comenta na definição das colunas que os valores vão de 3 a 7, ou seja, 0 é incorreto. Não realizamos a correção desses valores faltantes nesse exato momento, mas em breve voltaremos com eles.

## Exploração e Visualização dos Dados

Agora, na intenção de entender os dados e descobrir padrões, vamos realizar uma exploração e visualização dos dados! Para isso, vamos utilizar bibliotecas como Matplot e Seaborn. Nossa missão aqui é descobrir padrões, correlações e tendências nos dados. Vamos usar visualizações eficazes para comunicar os insights e justificar nossas futuras escolhas de features e modelos.

In [1303]:
# Definindo um estilo para nossos graficos e alterando o tamanho padrão das figuras geradas pelo Matplotlib
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

Para o começo da nossa análise, é importante saber a quantidade de músicas que são populares ou não e saber como está a distribuição entre elas. No gráfico gerado a seguir, é possível observar que existe uma divisão bem harmoniosa entre músicas populares e não populares.

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='popularity_target', data=df)
plt.title('Distribution of Song Popularity')
plt.xlabel('Popularity (0: Not Popular, 1: Popular)')
plt.ylabel('Count')
plt.show()

Partindo com nossa análise, vamos agora observar nossas variáveis numéricas e a relação entre elas. O Heatmap é um gráfico perfeito para isso que demonstra as relações entre as colunas. Resumindo bastante funciona da forma a seguir: 

Os valores de correlação variam de -1 a 1:

- **+1**: Correlação positiva perfeita. À medida que uma variável aumenta, a outra também aumenta.
- **-1**: Correlação negativa perfeita. À medida que uma variável aumenta, a outra diminui.
- **0**: Nenhuma correlação. As duas variáveis não afetam uma à outra.

In [ ]:
numerical_features = ['duration_ms', 'danceability', 'energy', 'key', 'loudness', 'speechiness', 
                      'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
plt.figure(figsize=(12, 10))
sns.heatmap(df[numerical_features].corr(), annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap of Numerical Features')
plt.show()

Realizando a análise do Heatmap acima, é possível identificar alguns padrões importantes sobre nossa base de dados.

1. Correlação forte entre 'energy' e 'loudness': Existe uma correlação positiva forte (0.76) entre 'energy' e 'loudness', indicando que músicas mais enérgicas tendem a ser mais altas.

2. Correlação negativa entre 'acousticness' e 'energy': Há uma forte correlação negativa (-0.73) entre 'acousticness' e 'energy', sugerindo que músicas mais acústicas tendem a ser menos enérgicas.

3. Correlação negativa entre 'acousticness' e 'loudness': Similarmente, existe uma correlação negativa moderada (-0.59) entre 'acousticness' e 'loudness', indicando que músicas acústicas tendem a ser menos altas.

4. Correlação positiva entre 'danceability' e 'valence': Há uma correlação positiva moderada (0.48) entre 'danceability' e 'valence', sugerindo que músicas mais dançantes tendem a ter um tom emocional mais positivo.

5. Correlação negativa entre 'instrumentalness' e 'loudness': Observa-se uma correlação negativa moderada (-0.43) entre 'instrumentalness' e 'loudness', indicando que músicas mais instrumentais tendem a ser menos altas.

6. Pouca correlação com 'duration_ms': A 'duration_ms' tem correlações fracas com a maioria das outras características, sugerindo que o comprimento da música não está fortemente relacionado com suas outras propriedades acústicas.

7. Correlações fracas com 'key': A 'key' tem correlações muito fracas com outras características, indicando que não há uma relação forte entre a tonalidade e outros aspectos musicais neste conjunto de dados.

8. Correlações moderadas com 'valence': A 'valence' tem correlações moderadas positivas com 'danceability' (0.48) e 'energy' (0.26), sugerindo que músicas mais positivas tendem a ser mais dançantes e enérgicas.

9. Pouca correlação entre 'tempo' e outras características: O 'tempo' da música tem correlações relativamente fracas com outras características, com a mais forte sendo com 'energy' (0.24).

10. Correlação fraca entre 'speechiness' e outras características: 'Speechiness' tem correlações geralmente fracas com outras características, com a mais notável sendo com 'liveness' (0.21).

11. 'Liveness' tem correlações fracas: A característica 'liveness' não mostra correlações fortes com nenhuma outra característica, sugerindo que é relativamente independente das outras propriedades musicais.

Saber desses padrão será importante para quando começarmos a selecionar as features para o treinamento do nosso modelo! Porque aqui é possível observar que alguns colunas estão extremamente ligadas com outras.

Agora, queremos entender como diferentes características musicais se relacionam com a popularidade das músicas. Por isso, vamos criar alguns gráficos para realizar essa visualização.

Com essa análise, vamos identificar quais características musicais têm maior influência na popularidade de uma música.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness']
for i, feature in enumerate(features):
    sns.boxplot(x='popularity_target', y=feature, data=df, ax=axes[i//3][i%3])
    axes[i//3][i%3].set_title(f'{feature.capitalize()} by Popularity')
plt.tight_layout()
plt.show()

Observando o gráfico acima, conseguimos ter alguns insights.

1. Danceability (Dançabilidade):

Há uma ligeira tendência de músicas populares (1) terem maior dançabilidade.
A diferença é pequena, mas notável, o que sugere que músicas mais dançantes têm uma probabilidade um pouco maior de serem populares.

2.Instrumentalness (Instrumentalidade):

Músicas populares tendem a ter menor instrumentalidade.
Isso sugere que músicas com vocais são geralmente mais populares do que músicas puramente instrumentais.

Agora, para entender quais gêneros proporcionam mais sucessos às músicas, vamos criar um gráfico para visualizar os 10 melhores gêneros em termos de músicas populares.

In [ ]:
top_genres = df['track_genre'].value_counts().nlargest(10)
plt.figure(figsize=(12, 6))
sns.barplot(x=top_genres.index, y=top_genres.values)
plt.title('Top 10 Genres by Count')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
sns.scatterplot(data=df, x='energy', y='danceability', hue='popularity_target', palette='viridis')
plt.title('Energy vs. Danceability (Colored by Popularity)')
plt.show()

Análisando o gráfico acima, podemos observar que a popularidade não parece ser exclusivamente determinada por altos níveis de energia e dançabilidade, já que músicas populares existem em diversos níveis de energia e dançabilidade, sugerindo que outros fatores também influenciam a popularidade.

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(data=df, x='duration_ms', bins=50, kde=True)
plt.title('Distribution of Song Durations')
plt.xlabel('Duration (ms)')
plt.show()

Observando a duração das músicas, podemos observar que há uma concentração muito alta de músicas com duração menor. Também observamos que existem músicas com durações extremamente longas. Estes podem ser álbuns inteiros, performances ao vivo, ou erros nos dados.

- A assimetria da distribuição pode afetar análises estatísticas, sendo necessário considerar transformações ou métodos robustos. Em breve voltaremos nesse tópico ao tratar os dados.

Já o gráfico de barras empilhadas abaixo apresenta uma comparação visual da proporção de conteúdo explícito entre músicas populares e não populares 

In [ ]:
explicit_by_popularity = df.groupby('popularity_target')['explicit'].value_counts(normalize=True).unstack()
explicit_by_popularity.plot(kind='bar', stacked=True)
plt.title('Proportion of Explicit Content by Popularity')
plt.xlabel('Popularity')
plt.ylabel('Proportion')
plt.legend(title='Explicit', loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()
plt.show()

Observando o gráfico acima, podemos observar que:

1. A maioria das músicas, tanto populares quanto não populares, não contém conteúdo explícito.
2. Há uma ligeira tendência de músicas populares terem uma proporção um pouco maior de conteúdo explícito em comparação com as não populares.
3. A presença de conteúdo explícito não parece ser um fator determinante para a popularidade de uma música, dada a pequena diferença observada.

Agora para fechar nossa análise, vamos comparar a distribuição da duração das músicas entre as categorias de popularidade.

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxenplot(x='popularity_target', y='duration_ms', data=df)
plt.title('Song Duration Distribution by Popularity')
plt.xlabel('Popularity (0: Not Popular, 1: Popular)')
plt.ylabel('Duration (ms)')
plt.show()

Observando o gráfico acima, vemos que:

1. As medianas de duração para músicas populares e não populares são muito semelhantes, sugerindo que a duração por si só não é um forte indicador de popularidade.
2. A dispersão (representada pelo tamanho das caixas) é ligeiramente menor para músicas populares, indicando uma maior consistência na duração dessas faixas.
3. Existem muitos outliers em ambas as categorias, representando músicas com durações excepcionalmente longas.
4. As músicas populares parecem ter menos outliers extremos em comparação com as não populares, especialmente na faixa de duração mais longa.
5. A maioria das músicas, independentemente da popularidade, tem duração dentro de um intervalo relativamente estreito, como evidenciado pelo tamanho das caixas do boxplot.
6. Há uma leve tendência de músicas populares terem durações um pouco mais curtas, observável pela posição ligeiramente inferior da caixa para músicas populares.

Os insights mais importantes que encontramos em nossa análise incluem: a ligeira tendência de músicas populares terem maior energia e dançabilidade; a predominância de conteúdo não explícito em ambas as categorias de popularidade, com uma sutil inclinação para mais conteúdo explícito em músicas populares; e a observação de que a duração das músicas não é um forte indicador de popularidade, embora músicas populares tendam a ter durações mais consistentes. Além disso, notamos correlações significativas entre certas características musicais, como a relação positiva entre energia e volume (loudness), e negativa entre acústica e energia.

Ainda é muito cedo para entrarmos na escolha das features para nosso modelo, mas já é possível observar que não existe uma coluna em específica que determina a popularidade das músicas, com isso, é bastante provável que vamos utilizar todas as colunas para treinar nosso modelo, excluindo apenas as colunas de identificação.

## Formulação de Hipóteses

#### Hipótese 01: Influência da dançabilidade na era do TikTok

Com a popularização do TikTok, onde coreografias virais são frequentes, músicas mais dançantes podem ter maior probabilidade de se tornarem populares. Esta hipótese sugere uma correlação positiva entre a dançabilidade de uma música e sua popularidade.

In [ ]:
plt.subplot(2, 2, 1)
sns.boxplot(x='popularity_target', y='danceability', data=df)
plt.title('Dançabilidade vs. Popularidade')
plt.xlabel('Popularidade (0: Não Popular, 1: Popular)')
plt.ylabel('Dançabilidade')

É possível observar que a média de músicas dançantes é um pouco mais alto para músicas populares, mas a diferença não é tão grande assim, logo, não indica extrema importância se a música é dançante ou não para ela ser popular.

#### Hipótese 02: Conteúdo explícito em músicas de temática triste

Músicas categorizadas como "sad" ou emocionalmente intensas podem ter uma maior tendência a conter conteúdo explícito.

In [ ]:
plt.subplot(2, 2, 2)
sad_genres = ['sad', 'melancholic', 'emotional']  # Ajuste conforme necessário
df['is_sad'] = df['track_genre'].isin(sad_genres)
sns.barplot(x='is_sad', y='explicit', data=df)
plt.title('Conteúdo Explícito em Músicas Tristes vs. Outras')
plt.xlabel('Gênero Triste')
plt.ylabel('Proporção de Conteúdo Explícito')
df = df.drop(columns='is_sad')

É possível observar que o conteúdo explícito está extremamente presente em músicas classificadas como 'sad', 'melancholic' ou 'emotional'. Comprovando nossa hipótese.

#### Hipótese 03: Popularidade de músicas com temática melancólica

Músicas classificadas como 'sad' estão entre as mais populares, principalmente porque hoje a depressão é o problema do século.

In [ ]:
top_genres = df['track_genre'].value_counts().nlargest(10)
plt.figure(figsize=(12, 6))
sns.barplot(x=top_genres.index, y=top_genres.values)
plt.title('Top 10 Genres by Count')
plt.xticks(rotation=45, ha='right')
plt.show()

É possível ver que o gênero 'sad' é o segundo gênero com maior sucesso de músicas populares. Comprovando nossa hipótese.

#### Hipótese 04: Relação inversa entre instrumentalidade e popularidade

Músicas com alto grau de instrumentalidade (pouco ou nenhum vocal) podem tender a ser menos populares, possivelmente devido à preferência do público mainstream por músicas com letras e vozes.

In [ ]:
plt.subplot(2, 2, 4)
sns.boxplot(x='popularity_target', y='instrumentalness', data=df)
plt.title('Instrumentalidade vs. Popularidade')
plt.xlabel('Popularidade (0: Não Popular, 1: Popular)')
plt.ylabel('Instrumentalidade')

É possível observar que existem músicas populares e que são instrumentais, mas, em maioria, elas são classificadas como não populares. Comprovando nossa hipótese.

## Limpeza e Tratamento de Valores Nulos

### Tratamento de valores duplicados

Fazer a limpeza de valores duplicados é importantes pois valores duplicados podem prejudicar no aprendizado do nosso algoritmo. Para realizar a limpeza, vamos primeiro checar se existem valores duplicados e caso existirem, vamos excluir todos.

In [ ]:
duplicated_values = df[df.duplicated()]
print(len(duplicated_values))

Como podemos ver acima, não existem valores duplicados. Assim, seguiremos com nossa análise

### Tratamento de missing values

Missing values são valores = null

É importante verificar a existência deles e resolver os valores faltantes caso existam.

In [ ]:
df.isnull().sum()

Como podemos ver acima, também não existem valores faltantes.

Agora, vamos verificar valores que são iguais a 0. Como identificamos lá no começo desse notebook, já sabemos de alguns valores que não deveriam estar iguais a 0. Também vamos procurar por novos valores iguais a 0 e que não deveriam estar assim, caso existam, vamos resolver.

In [ ]:
zero_count = (df == 0).sum()
print(zero_count)

Como podemos ver acima, as colunas 'time_signature' e 'tempo' possuem valores iguais a 0 e elas não deveriam estar assim:

1. Tempo:

- Representa a velocidade da música em batidas por minuto (BPM).
- Um valor 0 significaria que não há batidas, o que é musicalmente impossível.
- Tempos típicos variam de cerca de 60 BPM (lento) a 200 BPM (muito rápido).


2. Time signature (fórmula de compasso):

- Indica quantas batidas há por compasso e qual nota representa uma batida.
- É escrita como uma fração, por exemplo, 4/4 ou 3/4.
- Um valor 0 não faria sentido, pois implicaria em nenhuma batida por compasso.

In [ ]:
df['time_signature'].value_counts()

In [ ]:
print(df['tempo'].describe(), df['tempo'].value_counts())

Para resolver esses 2 problemas, vamos: 

- Substituir os valores de 'time_signature' pela moda, que é 4
- Substituir os valores de 'tempo' pela mediana

In [1321]:
tempo_median = df['tempo'].median()
df_treating_data = df.copy()
df_treating_data['tempo'] = df_treating_data['tempo'].replace(0,tempo_median)
df_treating_data['time_signature'] = df_treating_data['time_signature'].replace(0,4)

### Identificação de outliers e correção

As colunas que vamos visualizar e depois tratar os outliers são as colunas: 'duration_ms', 'loudness' e 'tempo'.

Realizei a escolha dessas três colunas porque todas as outras colunas numéricas se constituem de valores que vâo de 0 a 1. Algumas até possuem outliers mas são features importantes e que quero manter para preservar a pureza dos dados em features que vão de 0 a 1. Mais tarde, vamos minimizar os impactos que esses outliers podem causar com o standardScaler

Agora partindo para a identificação dos outliers que escolhemos, vamos começar visualizando em boxplot os possíveis outliers.

In [ ]:
sns.boxplot(data=df_treating_data, y='duration_ms')
plt.show()

Observando o gráfico acima, é possívei ver que 'duration_ms' possui vários outliers, vamos guardar essa informação e em breve corrigir eles.

Agora fazendo a análise dos outliers de 'loudness', vamos desenhar outro boxplot para realizar essa visualização:

In [ ]:
sns.boxplot(data=df_treating_data, y='loudness')
plt.show()

Agora fazendo a análise dos outliers de 'tempo', vamos desenhar outro boxplot para realizar essa visualização:

In [ ]:
sns.boxplot(data=df_treating_data, y='tempo')
plt.show()

Após uma análise cuidadosa dos dados, incluindo a presença de outliers em colunas como 'duration_ms', 'loudness' e 'tempo', decidimos manter o dataset em seu estado original, sem realizar nenhuma transformação ou exclusão neste momento.

Esta decisão se baseia nas seguintes considerações:

1. Preservação da integridade dos dados: Cada característica de uma música, mesmo aquelas que parecem estatisticamente atípicas, pode conter informações valiosas sobre a natureza única da faixa e potencialmente sobre sua popularidade.

2. Flexibilidade para análises futuras: Manter todos os dados em seu estado original nos proporciona máxima flexibilidade para realizar diversos tipos de análises e experimentos posteriormente.

3. Seleção de features posterior: Decidimos adiar a seleção de features para uma etapa posterior do processo. Isso nos permitirá tomar decisões mais informadas sobre quais características são mais relevantes para nosso modelo, baseando-nos em análises mais aprofundadas e experimentos iniciais.

4. Captura de padrões complexos: No domínio musical, o que pode parecer um outlier ou uma característica irrelevante pode, na verdade, ser parte de um padrão mais complexo que contribui para a popularidade de uma faixa.

5. Abordagem holística: Ao manter todos os dados, estamos adotando uma visão holística do problema, permitindo que nosso modelo potencialmente descubra relações e padrões que poderíamos não antecipar inicialmente.

6. Desafio realista para o modelo: Trabalhar com dados não processados cria um cenário mais próximo do mundo real, desafiando nosso modelo a lidar com a complexidade e variabilidade inerentes aos dados musicais.

Ao adiar a seleção de features, estamos nos dando a oportunidade de explorar diversas abordagens de modelagem e técnicas de seleção de características. Isso pode incluir métodos como:

- Análise de correlação entre features e a variável alvo (popularidade)
- Técnicas de seleção de features baseadas em importância (como as fornecidas por modelos de árvore)
- Métodos de redução de dimensionalidade (como PCA ou t-SNE)
- Testes de diferentes combinações de features em modelos preliminares

Esta abordagem nos permite ser mais metódicos e baseados em evidências em nossas decisões sobre quais características incluir em nosso modelo final. Também nos dá a flexibilidade de adaptar nossa estratégia conforme ganhamos mais insights sobre os dados e o problema em questão.

Nosso próximo passo será iniciar uma análise exploratória mais aprofundada dos dados, buscando entender melhor as relações entre as diferentes características e a popularidade das músicas. Isso nos ajudará a informar nossas decisões futuras sobre seleção de features e escolha de modelos.

## Codificação de Variáveis Categóricas

### Importância da Codificação

A codificação de variáveis categóricas é um passo crucial no pré-processamento de dados para modelos de machine learning. Sua importância se deve a vários fatores:

1. **Compatibilidade com algoritmos**: Muitos algoritmos de ML trabalham apenas com valores numéricos. A codificação transforma dados categóricos em formatos numéricos compatíveis.

2. **Preservação de informações**: Uma codificação eficaz preserva a informação contida nas categorias, permitindo que o modelo aprenda padrões relevantes.

3. **Melhoria do desempenho**: Uma codificação adequada pode melhorar significativamente o desempenho do modelo, capturando relações importantes nos dados.

4. **Tratamento de novas categorias**: Ajuda a lidar com categorias não vistas durante o treinamento, um problema comum em dados do mundo real.

### Explicação das Funções de Codificação

#### Função `normalize_and_encode_dataset`

Esta função é usada para codificar e normalizar o conjunto de dados de treinamento.

##### Processo:

1. **Target Encoding**: 
   - Utiliza validação cruzada K-Fold para evitar overfitting.
   - Para cada coluna, calcula a média do target para cada categoria.
   - Substitui cada categoria pelo valor médio correspondente do target.

2. **Tratamento de Valores Ausentes**:
   - Usa a média global do target para categorias não vistas.

3. **Normalização**:
   - Aplica StandardScaler para normalizar os valores codificados.

4. **Armazenamento**:
   - Salva os dicionários de codificação e os scalers para uso posterior.

#### Função `apply_normalize_and_encode`

Esta função é usada para aplicar a codificação e normalização ao conjunto de teste.

##### Processo:

1. **Aplicação da Codificação**:
   - Usa o dicionário de codificação gerado na função anterior.
   - Mapeia as categorias para seus valores codificados.

2. **Tratamento de Novas Categorias**:
   - Usa a média global para categorias não vistas no treinamento.

3. **Normalização**:
   - Aplica os mesmos scalers usados no conjunto de treinamento.

### Vantagens desta Abordagem

1. **Redução de Overfitting**: O uso de validação cruzada na codificação do conjunto de treinamento ajuda a prevenir overfitting.

2. **Consistência**: Garante que a codificação seja consistente entre os conjuntos de treinamento e teste.

3. **Flexibilidade**: Pode lidar com novas categorias no conjunto de teste.

4. **Normalização Integrada**: Combina codificação e normalização em um único processo.

In [1325]:
def normalize_and_encode_dataset(df, target, n_splits=5):
    encoded_df = df.copy()
    encoding_dict = {}
    scaler_dict = {}

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
   
    for column in df.columns:
        if column == target:
            continue
       
        encoded = np.zeros(len(df))
        column_dict = {}
       
        # Target encoding
        for train_idx, val_idx in kf.split(df):
            target_means = df.iloc[train_idx].groupby(column)[target].mean()
            encoded[val_idx] = df[column].iloc[val_idx].map(target_means)
           
            column_dict.update(target_means.to_dict())
       
        global_mean = df[target].mean()
        encoded = np.where(np.isnan(encoded), global_mean, encoded)
       
        column_dict['_global_mean'] = global_mean
        encoding_dict[column] = column_dict
       
        # Normalization
        scaler = StandardScaler()
        normalized = scaler.fit_transform(encoded.reshape(-1, 1)).flatten()
       
        encoded_df[f"{column}_encoded_normalized"] = normalized
        scaler_dict[column] = scaler
   
    return encoded_df, encoding_dict, scaler_dict


def apply_normalize_and_encode(df, encoding_dict, scaler_dict):
    encoded_df = df.copy()
   
    for column, column_dict in encoding_dict.items():
        if column in df.columns:
            global_mean = column_dict['_global_mean']
           
            # Apply target encoding
            encoded = df[column].map(column_dict)
            encoded = encoded.fillna(global_mean)
           
            # Apply normalization
            scaler = scaler_dict[column]
            normalized = scaler.transform(encoded.values.reshape(-1, 1)).flatten()
           
            encoded_df[f"{column}_encoded_normalized"] = normalized
        else:
            print(f"Warning: Column '{column}' not found in the input DataFrame.")
   
    return encoded_df

## Seleção de Features

Para nosso modelo de previsão de popularidade musical, selecionamos todas as colunas disponíveis, exceto 'track_id' e 'track_unique_id'. Esta abordagem abrangente nos permite capturar uma ampla gama de informações sobre cada faixa. As features selecionadas incluem:

1. Metadados: artists, album_name, track_name
2. Características de áudio: duration_ms, explicit, danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, time_signature
3. Categoria: track_genre

Justificativa: Cada uma dessas features pode contribuir de maneira única para a popularidade de uma música. Por exemplo, o artista e o álbum podem influenciar a exposição inicial da música, enquanto características de áudio como danceability e energy podem afetar a recepção do público.

## Engenharia de Features

Para aumentar o poder preditivo do nosso modelo, criamos novas features usando várias técnicas de engenharia. As principais funções utilizadas são:

1. `create_audio_interactions`:
   - Cria interações entre features de áudio existentes (ex: energy_danceability).
   - Gera features compostas como 'intensidade' e 'suavidade'.
   - Discretiza features contínuas em bins.
   Justificativa: Captura relações não lineares entre as features de áudio, que podem ser cruciais para entender a popularidade.

2. `frequency_encoding`:
   - Codifica variáveis categóricas com base em sua frequência.
   Justificativa: Ajuda o modelo a entender a raridade ou comunalidade de artistas, álbuns e gêneros, que pode ser um fator na popularidade.

3. `genre_popularity_encoding`:
   - Cria features baseadas na popularidade média, mediana e variabilidade de cada gênero.
   Justificativa: Fornece ao modelo informações sobre a tendência de popularidade de diferentes gêneros musicais.

4. `create_genre_hierarchy_features`:
   - Agrupa gêneros em clusters baseados em suas características de áudio.
   Justificativa: Permite que o modelo capture similaridades entre gêneros que podem não ser evidentes apenas pelo nome.

5. `create_genre_cooccurrence_features`:
   - Cria features baseadas na co-ocorrência de gêneros para artistas.
   Justificativa: Captura a versatilidade dos artistas e como isso pode afetar a popularidade.

6. `create_genre_diversity_features`:
   - Calcula a diversidade de gêneros para cada artista.
   Justificativa: Mede o quão diversificado é o repertório de um artista, o que pode influenciar sua popularidade.

7. `create_genre_crossover_score`:
   - Calcula o quão típica uma música é para seu gênero declarado.
   Justificativa: Músicas que se desviam significativamente das normas de seu gênero podem ter impactos únicos na popularidade.

8. `create_safe_genre_features`:
   - Cria features de gênero que são seguras de usar mesmo quando novos gêneros aparecem nos dados de teste.
   Justificativa: Garante que o modelo possa lidar com gêneros não vistos durante o treinamento.

9. `create_features`:
   - Cria várias features derivadas, incluindo interações, razões e categorizações.
   Justificativa: Fornece ao modelo mais informações derivadas que podem ser relevantes para prever popularidade.

10. `treat_nan_values`:
    - Preenche valores ausentes nas features.
    Justificativa: Garante que todas as observações possam ser usadas no treinamento e previsão.

Ao aplicar essas técnicas de engenharia de features, expandimos significativamente nosso conjunto de features. Isso permite que nosso modelo capture relações mais complexas e sutis nos dados, potencialmente melhorando sua capacidade de prever a popularidade das músicas com maior precisão.

In [1326]:
def create_audio_interactions(train_df, test_df):
    audio_features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
    
    for df in [train_df, test_df]:

        df['energy_danceability'] = df['energy'] * df['danceability']
        df['loudness_energy_ratio'] = df['loudness'] / (df['energy'] + 1e-5)
        df['valence_energy_ratio'] = df['valence'] / (df['energy'] + 1e-5)
        df['speechiness_instrumentalness_ratio'] = df['speechiness'] / (df['instrumentalness'] + 1e-5)
        
        df['intensity'] = (df['energy'] + df['loudness'] + df['tempo']) / 3
        df['mellowness'] = (df['acousticness'] + df['instrumentalness'] + (1 - df['energy'])) / 3
        df['complexity'] = (df['speechiness'] + df['instrumentalness'] + df['liveness']) / 3
        
        binner = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
        binned_features = binner.fit_transform(df[audio_features])
        
        for i, feature in enumerate(audio_features):
            df[f'{feature}_binned'] = binned_features[:, i]
        
        df['energy_danceability_binned'] = df['energy_binned'] * df['danceability_binned']
        df['loudness_tempo_binned'] = df['loudness_binned'] * df['tempo_binned']

    return train_df, test_df

In [1327]:
def frequency_encoding(train_df, test_df, columns=['artists', 'album_name', 'track_genre']):

    all_data = pd.concat([train_df, test_df], axis=0, ignore_index=True)
    
    for col in columns:
        freq_enc = all_data[col].value_counts(normalize=True)
        train_df[f'{col}_freq'] = train_df[col].map(freq_enc)
        test_df[f'{col}_freq'] = test_df[col].map(freq_enc)
        

        min_freq = freq_enc.min()
        train_df[f'{col}_freq'] = train_df[f'{col}_freq'].fillna(min_freq)
        test_df[f'{col}_freq'] = test_df[f'{col}_freq'].fillna(min_freq)
    
    return train_df, test_df

In [1328]:
def genre_popularity_encoding(train_df, test_df, target_column='popularity_target'):
    
    genre_stats = train_df.groupby('track_genre')[target_column].agg(['mean', 'median', 'std', 'count'])
    genre_stats['popularity_percentile'] = genre_stats['mean'].rank(pct=True)
    
    def apply_encoding(df):

        df['genre_mean_popularity'] = df['track_genre'].map(genre_stats['mean'])
        
        df['genre_median_popularity'] = df['track_genre'].map(genre_stats['median'])
        
        df['genre_popularity_percentile'] = df['track_genre'].map(genre_stats['popularity_percentile'])
        
        df['genre_popularity_variability'] = df['track_genre'].map(genre_stats['std'] / genre_stats['mean'])
        
        df['genre_track_count_log'] = df['track_genre'].map(np.log1p(genre_stats['count']))
        
        global_mean = genre_stats['mean'].mean()
        global_median = genre_stats['median'].median()
        global_percentile = 0.5  # middle percentile
        global_variability = (genre_stats['std'] / genre_stats['mean']).mean()
        global_count_log = np.log1p(genre_stats['count'].mean())
        
        df['genre_mean_popularity'] = df['genre_mean_popularity'].fillna(global_mean)
        df['genre_median_popularity'] = df['genre_median_popularity'].fillna(global_median)
        df['genre_popularity_percentile'] = df['genre_popularity_percentile'].fillna(global_percentile)
        df['genre_popularity_variability'] = df['genre_popularity_variability'].fillna(global_variability)
        df['genre_track_count_log'] = df['genre_track_count_log'].fillna(global_count_log)
        
        return df
    
    encoded_train = apply_encoding(train_df.copy())
    encoded_test = apply_encoding(test_df.copy())
    
    return encoded_train, encoded_test

In [1329]:
def create_genre_hierarchy_features(train_df, test_df, n_clusters=10):

    all_data = pd.concat([train_df, test_df], axis=0, ignore_index=True)
    
    audio_features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
    genre_features = all_data.groupby('track_genre')[audio_features].mean()
    
    clustering = AgglomerativeClustering(n_clusters=n_clusters)
    genre_clusters = clustering.fit_predict(genre_features)
    
    genre_cluster_dict = dict(zip(genre_features.index, genre_clusters))
    
    for df in [train_df, test_df]:
        df['genre_cluster'] = df['track_genre'].map(genre_cluster_dict)
        
        cluster_dummies = pd.get_dummies(df['genre_cluster'], prefix='genre_cluster')
        df = pd.concat([df, cluster_dummies], axis=1)
    
    return train_df, test_df

In [1330]:
def create_genre_cooccurrence_features(train_df, test_df, n_components=10):

    all_data = pd.concat([train_df, test_df], axis=0, ignore_index=True)
    
    artist_genres = all_data.groupby('artists')['track_genre'].apply(lambda x: ' '.join(set(x))).reset_index()
    
    vectorizer = CountVectorizer()
    genre_matrix = vectorizer.fit_transform(artist_genres['track_genre'])
    
    svd = TruncatedSVD(n_components=n_components, random_state=42)
    genre_cooccurrence = svd.fit_transform(genre_matrix)
    
    genres = vectorizer.get_feature_names_out()
    genre_cooccur_dict = {genre: cooccur for genre, cooccur in zip(genres, genre_cooccurrence)}
    
    def safe_get_cooccur(genre):
        cooccur = genre_cooccur_dict.get(genre)
        if cooccur is None or np.isscalar(cooccur):
            return np.zeros(n_components)
        return cooccur
    
    for df in [train_df, test_df]:
        cooccur_vectors = df['track_genre'].apply(safe_get_cooccur)
        for i in range(n_components):
            df[f'genre_cooccur_{i}'] = cooccur_vectors.apply(lambda x: x[i])
    
    return train_df, test_df

In [1331]:
def custom_entropy(probabilities):

    probabilities = probabilities[probabilities > 0]
    return -np.sum(probabilities * np.log(probabilities))

def create_genre_diversity_features(train_df, test_df):

    all_data = pd.concat([train_df, test_df], axis=0, ignore_index=True)
    
    artist_genre_counts = all_data.groupby('artists')['track_genre'].value_counts().unstack(fill_value=0)
    artist_genre_probs = normalize(artist_genre_counts, norm='l1', axis=1)
    artist_genre_entropy = np.apply_along_axis(custom_entropy, 1, artist_genre_probs)
    artist_genre_count = artist_genre_counts.sum(axis=1)
    
    artist_diversity_dict = dict(zip(artist_genre_counts.index, artist_genre_entropy))
    artist_genre_count_dict = dict(zip(artist_genre_count.index, artist_genre_count.values))
    
    for df in [train_df, test_df]:
        df['artist_genre_diversity'] = df['artists'].map(artist_diversity_dict)
        df['artist_genre_count'] = df['artists'].map(artist_genre_count_dict)
        df['artist_genre_diversity_ratio'] = df['artist_genre_diversity'] / np.log(df['artist_genre_count'])
    
    return train_df, test_df

In [1332]:
def create_genre_crossover_score(train_df, test_df):
    audio_features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
    
    genre_audio_avg = train_df.groupby('track_genre')[audio_features].mean()
    
    def calculate_crossover_score(row):
        genre_avg = genre_audio_avg.loc[row['track_genre']]
        return np.mean([abs(row[feat] - genre_avg[feat]) for feat in audio_features])
    
    for df in [train_df, test_df]:
        df['genre_crossover_score'] = df.apply(calculate_crossover_score, axis=1)
    
    return train_df, test_df

In [1333]:
def create_safe_genre_features(train_df, test_df):

    all_data = pd.concat([train_df, test_df], axis=0, ignore_index=True)
    
    audio_features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
    
    genre_stats = all_data.groupby('track_genre')[audio_features].agg(['mean', 'std'])
    genre_stats.columns = [f'genre_{col[0]}_{col[1]}' for col in genre_stats.columns]
    
    genre_freq = all_data['track_genre'].value_counts(normalize=True).to_dict()
    
    def apply_genre_encodings(df):

        df = df.merge(genre_stats, on='track_genre', how='left')
        
        df['genre_frequency'] = df['track_genre'].map(genre_freq)
        
        for feature in audio_features:
            df[f'{feature}_genre_deviation'] = df[feature] - df[f'genre_{feature}_mean']
        
        df['genre_typicality'] = np.mean([
            1 - abs(df[f'{feature}_genre_deviation'] / df[f'genre_{feature}_std'])
            for feature in audio_features
        ], axis=0)
        
        return df
    
    train_df = apply_genre_encodings(train_df)
    test_df = apply_genre_encodings(test_df)
    
    return train_df, test_df

In [1334]:
def create_features(df):

    df['energy_loudness_interaction'] = df['energy'] * df['loudness']
    df['danceability_tempo_interaction'] = df['danceability'] * df['tempo']
    df['acousticness_instrumentalness_interaction'] = df['acousticness'] * df['instrumentalness']

    df['energy_danceability_ratio'] = df['energy'] / (df['danceability'] + 1e-5)
    df['valence_arousal_ratio'] = df['valence'] / (np.sqrt(df['energy']**2 + df['danceability']**2) + 1e-5)

    df['duration_minutes'] = df['duration_ms'] / 60000
    df['duration_tempo_ratio'] = df['duration_minutes'] / (df['tempo'] + 1e-5)

    df['audio_complexity'] = (df['instrumentalness'] + df['acousticness'] + df['speechiness']) / 3

    audio_features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
    df['audio_feature_range'] = df[audio_features].max(axis=1) - df[audio_features].min(axis=1)

    df['tempo_category'] = pd.cut(df['tempo'], bins=[0, 60, 90, 120, 150, np.inf], labels=[0, 1, 2, 3, 4])

    df['key_mode_interaction'] = df['key'] * df['mode']

    def custom_entropy(x):
        probs = x.value_counts(normalize=True)
        return -np.sum(probs * np.log2(probs + 1e-10))
    
    df['audio_feature_entropy'] = df[audio_features].apply(lambda x: custom_entropy(pd.cut(x, bins=10)), axis=1)

    df['genre_encoded'] = df['track_genre'].astype('category').cat.codes

    df['artist_encoded'] = df['artists'].astype('category').cat.codes

    df['album_encoded'] = df['album_name'].astype('category').cat.codes

    numerical_features = audio_features + ['duration_ms', 'key', 'time_signature']
    scaler = StandardScaler()
    df[numerical_features] = scaler.fit_transform(df[numerical_features])

    return df

In [1335]:
def treat_nan_values(df):

    df_treated = df.copy()

    numeric_columns = df.select_dtypes(include=[np.number]).columns
    non_numeric_columns = df.select_dtypes(exclude=[np.number]).columns
    
    for col in numeric_columns:
        df_treated[col].fillna(df[col].median(), inplace=True)
    
    for col in non_numeric_columns:
        df_treated[col].fillna(df[col].mode()[0], inplace=True)
    
    return df_treated

Agora, criamos uma cópia da nossa base de dados e dropamos as colunas que não vamos usar

In [1336]:
new_df = df_treating_data.copy()
new_df = new_df.drop(columns=['track_unique_id', 'track_id'])

Agora, vamos aplicar a criação de novas features

In [ ]:
new_df, test_df = create_audio_interactions(new_df, df_test)
new_df, test_df = frequency_encoding(new_df, df_test)
new_df, test_df = genre_popularity_encoding(new_df, test_df)
new_df, test_df = create_genre_hierarchy_features(new_df, test_df)
new_df, test_df = create_genre_cooccurrence_features(new_df, test_df)
new_df, test_df = create_genre_diversity_features(new_df, test_df)
new_df, test_df = create_safe_genre_features(new_df, test_df)
new_df, test_df = create_genre_crossover_score(new_df, test_df)
new_df = treat_nan_values(new_df)
test_df = treat_nan_values(test_df)

Aqui, codificamos tudo e já criamos os dicionários para usar depois com nossa base de teste

In [1338]:
encoded_df, encoding_dict, scaler_dict = normalize_and_encode_dataset(new_df, 'popularity_target')

Com base de muito testes, as colunas abaixo são as melhores colunas para se excluir e manter a melhor acurária possível

In [1339]:
encoded_df = encoded_df.drop(columns=['artists', 'album_name', 'track_name', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre', 'genre_mean_popularity_encoded_normalized', 'genre_median_popularity_encoded_normalized', 'genre_popularity_percentile_encoded_normalized', 'genre_popularity_variability_encoded_normalized', 'genre_track_count_log_encoded_normalized', 'artists_freq_encoded_normalized', 'album_name_freq_encoded_normalized', 'track_genre_freq_encoded_normalized', 'genre_cooccur_0_encoded_normalized','genre_cooccur_1_encoded_normalized','genre_cooccur_2_encoded_normalized','genre_cooccur_3_encoded_normalized','genre_cooccur_4_encoded_normalized','genre_cooccur_5_encoded_normalized','genre_cooccur_6_encoded_normalized','genre_cooccur_7_encoded_normalized','genre_cooccur_8_encoded_normalized','genre_cooccur_9_encoded_normalized', 'artist_genre_diversity_encoded_normalized','artist_genre_count_encoded_normalized','artist_genre_diversity_ratio_encoded_normalized', 'genre_cluster_encoded_normalized', 'genre_crossover_score'])

Agora, vamos separar nossa base de dados para começar os testes

In [1340]:
X = encoded_df.drop('popularity_target', axis=1)
y = encoded_df['popularity_target']

In [ ]:
X_temp_df_no_genres, X_test_df_no_genres, y_temp_df_no_genres, y_test_df_no_genres = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_df_no_genres, X_val_df_no_genres, y_train_df_no_genres, y_val_df_no_genres = train_test_split(X_temp_df_no_genres, y_temp_df_no_genres, test_size=0.25, random_state=42)

print(f"Tamanho do conjunto de treino: {len(X_train_df_no_genres)}")
print(f"Tamanho do conjunto de validação: {len(X_val_df_no_genres)}")
print(f"Tamanho do conjunto de teste: {len(X_test_df_no_genres)}")

## Finetuning de Hiperparâmetros

O finetuning de hiperparâmetros é um processo crucial no desenvolvimento de modelos de machine learning. Ele envolve a otimização dos parâmetros que controlam o processo de aprendizagem do modelo, distintos dos parâmetros internos que o modelo aprende durante o treinamento.

### Importância

1. **Melhoria de Desempenho**: Hiperparâmetros bem ajustados podem melhorar significativamente a acurácia e generalização do modelo.
2. **Prevenção de Overfitting**: Ajuda a encontrar o equilíbrio entre complexidade do modelo e capacidade de generalização.
3. **Eficiência Computacional**: Otimiza o uso de recursos computacionais durante o treinamento.

### Grid Search

Grid Search é uma técnica de finetuning que envolve:

1. Definição de um conjunto de valores possíveis para cada hiperparâmetro.
2. Criação de todas as combinações possíveis desses valores.
3. Treinamento e avaliação do modelo para cada combinação.
4. Seleção da combinação que produz o melhor desempenho.

In [ ]:
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

hgbc_param_grid = {
    'max_iter': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_leaf': [1, 5, 10],
    'max_bins': [255, 100, 50]
}

def perform_grid_search(classifier, param_grid, X_train, y_train):
    grid_search = GridSearchCV(classifier, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    return grid_search

rf_classifier = RandomForestClassifier(random_state=42)
rf_grid_search = perform_grid_search(rf_classifier, rf_param_grid, X_train_df_no_genres, y_train_df_no_genres)

hgb_classifier = HistGradientBoostingClassifier(random_state=42)
hgb_grid_search = perform_grid_search(hgb_classifier, hgbc_param_grid, X_train_df_no_genres, y_train_df_no_genres)

print("Random Forest Best Parameters:")
print(rf_grid_search.best_params_)
print("Random Forest Best Accuracy:", rf_grid_search.best_score_)

print("\nHistogram Gradient Boosting Best Parameters:")
print(hgb_grid_search.best_params_)
print("Histogram Gradient Boosting Best Accuracy:", hgb_grid_search.best_score_)

rf_best = rf_grid_search.best_estimator_
hgb_best = hgb_grid_search.best_estimator_

rf_val_pred = rf_best.predict(X_val_df_no_genres)
hgb_val_pred = hgb_best.predict(X_val_df_no_genres)

print("\nRandom Forest Validation Accuracy:", accuracy_score(y_val_df_no_genres, rf_val_pred))
print("Histogram Gradient Boosting Validation Accuracy:", accuracy_score(y_val_df_no_genres, hgb_val_pred))

In [ ]:
gbc_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'min_samples_split': [2, 5, 10]
}

et_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

bc_param_grid = {
    'n_estimators': [10, 50, 100],
    'max_samples': [0.5, 0.7, 1.0],
    'max_features': [0.5, 0.7, 1.0],
    'bootstrap': [True, False],
    'bootstrap_features': [True, False]
}

def perform_grid_search(classifier, param_grid, X_train, y_train):
    grid_search = GridSearchCV(classifier, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    return grid_search

classifiers = [
    ("Gradient Boosting", GradientBoostingClassifier(random_state=42), gbc_param_grid),
    ("Extra Trees", ExtraTreesClassifier(random_state=42), et_param_grid),
    ("Bagging", BaggingClassifier(random_state=42), bc_param_grid)
]

results = {}

for name, classifier, param_grid in classifiers:
    print(f"\nPerforming grid search for {name}...")
    grid_search = perform_grid_search(classifier, param_grid, X_train_df_no_genres, y_train_df_no_genres)
    results[name] = grid_search

    print(f"{name} Best Parameters:")
    print(grid_search.best_params_)
    print(f"{name} Best Accuracy:", grid_search.best_score_)

    best_model = grid_search.best_estimator_
    val_pred = best_model.predict(X_val_df_no_genres)
    val_accuracy = accuracy_score(y_val_df_no_genres, val_pred)
    print(f"{name} Validation Accuracy:", val_accuracy)

print("\nSummary of Results:")
for name, grid_search in results.items():
    print(f"{name}:")
    print(f"  Best Parameters: {grid_search.best_params_}")
    print(f"  Best Cross-validation Accuracy: {grid_search.best_score_:.4f}")
    best_model = grid_search.best_estimator_
    val_pred = best_model.predict(X_val_df_no_genres)
    val_accuracy = accuracy_score(y_val_df_no_genres, val_pred)
    print(f"  Validation Accuracy: {val_accuracy:.4f}")
    print()

### Resultados do Finetuning de Hiperparâmetros

| Modelo                         | Melhores Parâmetros                                                                                              | Acurácia CV | Acurácia Validação |
|--------------------------------|------------------------------------------------------------------------------------------------------------------|-------------|---------------------|
| Random Forest                  | max_depth: None<br>min_samples_leaf: 1<br>min_samples_split: 2<br>n_estimators: 300                              | 0.9213      | 0.9238              |
| Histogram Gradient Boosting    | max_bins: 255<br>max_depth: 10<br>max_iter: 200<br>min_samples_leaf: 5                                           | 0.9192      | 0.9193              |
| Gradient Boosting              | learning_rate: 0.1<br>max_depth: 7<br>min_samples_split: 10<br>n_estimators: 200                                 | 0.9216      | 0.9228              |
| Extra Trees                    | max_depth: 30<br>min_samples_leaf: 1<br>min_samples_split: 5<br>n_estimators: 300                                | 0.9176      | 0.9212              |
| Bagging                        | bootstrap: False<br>bootstrap_features: False<br>max_features: 0.7<br>max_samples: 0.7<br>n_estimators: 100      | 0.9219      | 0.9237              |

Com o resultado, vamos criar nossos algoritmos

In [1352]:
rf_classifier = RandomForestClassifier(max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=300, random_state=42)
hist_gbc = HistGradientBoostingClassifier(max_bins=255, max_depth=10, max_iter=200, min_samples_leaf=5)
gbc = GradientBoostingClassifier(learning_rate=0.1, max_depth=7, min_samples_split=10, n_estimators=200, random_state=42)
et = ExtraTreesClassifier(max_depth=30, min_samples_leaf=1, min_samples_split=5, n_estimators=300, random_state=42)
bc = BaggingClassifier(bootstrap=False, bootstrap_features=False, max_features=0.7, max_samples=0.7, n_estimators=100, random_state=42)
mlp = MLPClassifier(activation='relu',alpha=0.0001,hidden_layer_sizes=(100,50), learning_rate='constant', solver='adam')

### Em busca da melhor `accuracy`

O Weighted Voting Classifier é uma técnica de ensemble learning que combina as previsões de múltiplos modelos de machine learning para fazer uma previsão final. Na votação ponderada (weighted voting), cada modelo individual recebe um peso que determina sua influência na decisão final.

#### Funcionamento

1. **Combinação de Modelos**: Vários modelos são treinados independentemente nos mesmos dados.

2. **Atribuição de Pesos**: Cada modelo recebe um peso baseado em sua confiabilidade ou performance.

3. **Votação Soft**: No modo 'soft', cada classificador fornece uma probabilidade para cada classe, em vez de apenas uma previsão de classe.

4. **Agregação Ponderada**: As probabilidades de cada modelo são multiplicadas por seus respectivos pesos e somadas.

5. **Decisão Final**: A classe com a maior soma ponderada de probabilidades é escolhida como a previsão final.

#### Vantagens

1. **Melhoria da Acurácia**: Combina as forças de diferentes modelos, potencialmente superando o desempenho de qualquer modelo individual.
2. **Redução de Overfitting**: A agregação de múltiplos modelos pode ajudar a reduzir o overfitting.
3. **Robustez**: Menos sensível a outliers ou dados ruidosos comparado a modelos individuais.

In [ ]:
weighted_voting_clf = VotingClassifier(
    estimators=[
        ('hist_gbc', hist_gbc),
        ('rf_classifier', rf_classifier),
        ('gbc', gbc),
        ('et', et),
        ('bc', bc),
    ],
    voting='soft',
    weights=[1, 1, 1, 2, 3]
)

weighted_voting_clf.fit(X_train_df_no_genres, y_train_df_no_genres)
y_pred_val = weighted_voting_clf.predict(X_val_df_no_genres)
y_pred_test = weighted_voting_clf.predict(X_test_df_no_genres)

print("Weighted Voting Classifier - Validation Accuracy:", accuracy_score(y_val_df_no_genres, y_pred_val))
print("Weighted Voting Classifier - Test Accuracy:", accuracy_score(y_test_df_no_genres, y_pred_test))
print("Weighted Voting Classifier - Validation Accuracy:", classification_report(y_val_df_no_genres, y_pred_val))
print("Weighted Voting Classifier - Test Accuracy:", classification_report(y_test_df_no_genres, y_pred_test))

#### Análise Geral

1. **Consistência**: As métricas são consistentes entre os conjuntos de validação e teste, indicando boa generalização do modelo.

2. **Equilíbrio de Classes**: O desempenho é equilibrado entre as classes 0 e 1, sugerindo que o modelo não é enviesado para uma classe específica.

3. **Alto Desempenho**: Todas as métricas estão acima de 0.92, indicando um excelente desempenho geral do modelo.

4. **Robustez**: A pequena diferença entre as métricas de validação e teste (menos de 0.5 pontos percentuais) sugere que o modelo é robusto e não está sofrendo de overfitting significativo.

Agora vamos aplicar tudo o que aplicamos no base de treino, para a base de testes. Depois salvar o resultado em CSV e fazer o envio da atividade.

In [1355]:
df_test_target_encoding = test_df.copy()
df_test_target_encoding = df_test_target_encoding.drop(columns=['track_unique_id', 'track_id'])

df_test_target_encoding['tempo'] = df_test_target_encoding['tempo'].replace(0,tempo_median)
df_test_target_encoding['time_signature'] = df_test_target_encoding['time_signature'].replace(0,4)

df_test_target_encoding = apply_normalize_and_encode(df_test_target_encoding, encoding_dict, scaler_dict)

df_test_target_encoding = df_test_target_encoding.drop(columns=['artists', 'album_name', 'track_name', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre', 'genre_mean_popularity_encoded_normalized', 'genre_median_popularity_encoded_normalized', 'genre_popularity_percentile_encoded_normalized', 'genre_popularity_variability_encoded_normalized', 'genre_track_count_log_encoded_normalized', 'artists_freq_encoded_normalized', 'album_name_freq_encoded_normalized', 'track_genre_freq_encoded_normalized', 'genre_cooccur_0_encoded_normalized','genre_cooccur_1_encoded_normalized','genre_cooccur_2_encoded_normalized','genre_cooccur_3_encoded_normalized','genre_cooccur_4_encoded_normalized','genre_cooccur_5_encoded_normalized','genre_cooccur_6_encoded_normalized','genre_cooccur_7_encoded_normalized','genre_cooccur_8_encoded_normalized','genre_cooccur_9_encoded_normalized', 'artist_genre_diversity_encoded_normalized','artist_genre_count_encoded_normalized','artist_genre_diversity_ratio_encoded_normalized', 'genre_cluster_encoded_normalized', 'genre_crossover_score'])

predict_target_encoding_stacking = weighted_voting_clf.predict(df_test_target_encoding)
result_stacking = pd.DataFrame({ 'track_unique_id': df_test['track_unique_id'], 'popularity_target': predict_target_encoding_stacking })
result_stacking.to_csv('result_weighed_11123.csv', index=False)